In [1]:
%pip install numpy
%pip install scikit-learn
%pip install keras
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl (377.1 MB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import numpy as np

In [3]:
class NetNode(object):
    """ Base class that represents a node in a neural net """

    def __init__(self):
        self.inputs = []
        self.weights = []
        self.value = None

In [4]:
class Network(object):
    """ Main class to construct ant train neural networks """

    def __init__(self, layers):
        """ 
        Parameters
        ----------
        layers:
            A list that represents the neurons and layers of the network.
            For example, [2, 3, 1] represents a network with 3 layers:
            - input layer: 2 neurons.
            - hidden layer: 3 neurons.
            - output layer: 1 neuron. 
        """
        self.net = [[NetNode() for _ in range(size)] for size in layers]

        sizes = len(layers)

        # Make connections
        for layer in range(1, sizes):
            for node in self.net[layer]:
                for unit in self.net[layer - 1]:
                    node.inputs.append(unit)
                    node.weights.append(0)

    def relu(self, z):
        """ Relu activation function """
        return max(0, z)

    def relu_prime(self, z):
        """ Derivative of relu activation function """
        return 1 if z > 0 else 0

    def predict(self, input_data):
        inputs = self.net[0]

        # Initialize inputs
        for v, n in zip(input_data, inputs):
            n.value = v

        # Forward step
        for layer in self.net[1:]:
            for node in layer:
                in_val = [n.value for n in node.inputs]
                unit_value = np.dot(in_val, node.weights)
                node.value = self.relu(unit_value)

        outputs = self.net[-1]
        return outputs.index(max(outputs, key=lambda node: node.value))

    def accuracy(self, examples):
        correct = 0

        for x_test, y_test in examples:
            prediction = self.predict(x_test)

            if (y_test[prediction] == 1):
                correct += 1

        return correct / len(examples)

    def backpropagation(self, eta, examples, epochs):
        inputs = self.net[0]
        outputs = self.net[-1]
        layer_size = len(self.net)

        # Initialize weights
        for layer in self.net[1:]:
            for node in layer:
                node.weights = [np.random.uniform()
                                for _ in range(len(node.weights))]

        for epoch in range(epochs):
            for x_train, y_train in examples:
                # Initialize inputs
                for value, node in zip(x_train, inputs):
                    node.value = value

                # Forward step
                for layer in self.net[1:]:
                    for node in layer:
                        in_val = [n.value for n in node.inputs]
                        unit_value = np.dot(in_val, node.weights)
                        node.value = self.relu(unit_value)

                # Initialize delta
                delta = [[] for _ in range(layer_size)]

                # Error for the MSE cost function
                err = [y_train[i] -
                       outputs[i].value for i in range(len(outputs))]

                delta[-1] = [self.relu_prime(outputs[i].value) * err[i]
                             for i in range(len(outputs))]

                # Backward step
                hidden_layers = layer_size - 2
                for i in range(hidden_layers, 0, -1):
                    layer = self.net[i]
                    n_layers = len(layer)

                    # Weights from the last layer
                    w = [[node.weights[l] for node in self.net[i + 1]]
                         for l in range(n_layers)]

                    delta[i] = [self.relu_prime(
                        layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]

                # Update weights
                for i in range(1, layer_size):
                    layer = self.net[i]
                    in_val = [node.value for node in self.net[i - 1]]
                    n_layers = len(self.net[i])
                    for j in range(n_layers):
                        layer[j].weights = np.add(
                            layer[j].weights, np.multiply(eta * delta[i][j], in_val))

            print(
                f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")


In [5]:
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [6]:
# import data to play with
iris_X, iris_y = datasets.load_iris(return_X_y=True)

In [7]:
# First 10 elements of input data
iris_X[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [8]:
# First 10 elements of output data
iris_y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
iris_x_normalized = normalize(iris_X, axis=0)

In [10]:
# Creating train and test data
'''
80% -- train data
20% -- test data
'''
X_train, X_test, y_train, y_test = train_test_split(
    iris_x_normalized, iris_y, test_size=0.2, shuffle=True)

In [11]:
# Convert classes from categorical ('Setosa', 'Versicolor', 'Virginica')
# to numerical (0, 1, 2) and then to one-hot encoded ([1, 0, 0], [0, 1, 0], [0, 0, 1]).
'''
[0]--->[1 0 0]
[1]--->[0 1 0]
[2]--->[0 0 1]
'''
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

In [12]:
examples = []
for i in range(len(X_train)):
    examples.append([X_train[i], y_train[i]])

In [13]:
net = Network([4, 7, 3])
net.backpropagation(0.1, examples, 500)

epoch 0/500 | total error=-0.0006958659248130998
epoch 1/500 | total error=-0.0005319071093649338
epoch 2/500 | total error=-0.0005397383769692787
epoch 3/500 | total error=-0.0005594327807716575
epoch 4/500 | total error=-0.0005804677519421653
epoch 5/500 | total error=-0.0006032550113053726
epoch 6/500 | total error=-0.0006303933223643703
epoch 7/500 | total error=-0.0006599099422758072
epoch 8/500 | total error=-0.0006805775540859987
epoch 9/500 | total error=-0.0006576345827536139
epoch 10/500 | total error=-0.0004980857787902433
epoch 11/500 | total error=-0.00028755117216201315
epoch 12/500 | total error=-5.218524392956355e-05
epoch 13/500 | total error=0.00021800852536617196
epoch 14/500 | total error=0.0004809730460671217
epoch 15/500 | total error=0.0007371776950278861
epoch 16/500 | total error=0.0009727947938690594
epoch 17/500 | total error=0.001198987646220199
epoch 18/500 | total error=0.0014162037456127965
epoch 19/500 | total error=0.0016185671892637874
epoch 20/500 | t

In [14]:
examples = []
for i in range(len(X_test)):
    examples.append([X_test[i], y_test[i]])


In [15]:
accuracy = net.accuracy(examples)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9666666666666667


In [16]:
prediction = net.predict(X_test[1])
print(f"Desired output: {y_test[1]}")
print(f"Index of output: {prediction}")

Desired output: [1. 0. 0.]
Index of output: 0
